## LangChain: Models, Prompts and Output Parsers
### Outline
- Direct API calls to Ollama
- API calls through LangChain:
    - Prompts
    - Models
    - Output parsers

## Ollama Install
[ollama docker-image | ollama ](https://ollama.com/blog/ollama-is-now-available-as-an-official-docker-image)

```bash
sudo nvidia-ctk runtime configure --runtime=docker
sudo systemctl restart docker
```
```bash
docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
docker exec -it ollama ollama run llama3
```

```bash
%pip install langchain-ollama
%pip install python-dotenv
%pip install langchain langchain-core
```

### Model

In [4]:
from langchain_ollama.llms import OllamaLLM
chat = OllamaLLM(temperature=0.0, model="llama3")
chat

OllamaLLM(model='llama3', temperature=0.0)

### Prompt Template

In [24]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [25]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [26]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [27]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [38]:
customer_style = """
English
"""

customer_email = """
你好嗎
"""

In [39]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [40]:
customer_response = chat.invoke(customer_messages)

In [41]:
print(customer_response)

The text you provided, `你好嗎`, is in Chinese.

Translated to English, it means:

"Hello? How are you?"


1. Prompts 可以很長和很仔細描述
2. 好的 Prompt 是可以被重複使用

特別是透過設計好的 prompt template，配合特定關鍵詞來執行類似「思考－行動－觀察」的邏輯流程（稱為 ReAct 框架）。

```
Question: ...
Thought: ...         ← 想法
Action: ...          ← 執行某個動作（如搜尋、查資料）
Observation: ...     ← 根據動作得到的觀察結果
```

## Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


In [48]:
response_schemas = [
    ResponseSchema(name="thought", description="The reasoning step"),
    ResponseSchema(name="action", description="The next action to take"),
    ResponseSchema(name="observation", description="Observation from the action"),
]


In [49]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [50]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"thought": string  // The reasoning step
	"action": string  // The next action to take
	"observation": string  // Observation from the action
}
```


In [ ]:
react_template="""
Answer the user's question step-by-step using ReAct format.

Caculator math. But except multiplication and div.

Output Example:

```json
{{
  "thought": "I need to compute 1 + 1",
  "action": "Calculate[1 + 1]",
  "observation": "2"
}}
```

Question: {question}

{format_instructions}
"""
# LangChain（底層是 Jinja-like 模板）允許你透過 {{ 與 }} 來逃逸 {}
prompt = ChatPromptTemplate.from_template(
    template=react_template)

print(prompt)

input_variables=['format_instructions', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], input_types={}, partial_variables={}, template='\nAnswer the user\'s question step-by-step using ReAct format.\n\nCaculator math. But except multiplication and div.\n\nOutput Example:\n\n```json\n{{\n  "thought": "I need to compute 1 + 1",\n  "action": "Calculate[1 + 1]",\n  "observation": "2"\n}}\n```\n\nQuestion: {question}\n\n{format_instructions}\n'), additional_kwargs={})]


In [73]:
customer_messages = prompt.format_messages(
                    question="What is number for 1 + 1",
                    format_instructions=format_instructions)

In [74]:
result = chat.invoke(customer_messages)

In [75]:
print(result)

Here is the answer in ReAct format:

```json
{
  "thought": "I need to compute 1 + 1",
  "action": "Calculate[1 + 1]",
  "observation": "2"
}
```

Let me know if you have any further questions!


In [80]:
output_dict = output_parser.parse(result)
type(output_dict)
print(output_dict.get('observation'))

2
